# OkCupid Profiles Topic Modeling and Analysis
- This notebook explores OkCupid dating profile bio descriptions.  We have a dataset of profiles containing name, age, location, and description.  The data was obtained using an API.  Here, we use a small sample of ~3400 observations.
- Let's start with exploratory summary stats, then see what clusters form when we apply topic modeling techniques.

In [10]:
import pandas as pd
# from sqlalchemy import create_engine
# import pymysql

In [12]:
# engine = create_engine("mysql+pymysql://[redacted]:[redacted]@[redacted]/[redacted]?host=localhost?port=3306")
# conn = engine.connect()
# sql = 'SELECT * from profile'
# df = pd.read_sql(sql, conn)

df = pd.read_csv('data/OKC_profiles.csv')